## Not implemented in IPYNB
Refer to 
https://github.com/jalkestrup/llm2vec-dtu/blob/main/experiments/run_supervised.py

In [1]:
import torch
import transformers
from tqdm import tqdm
import os
from accelerate import Accelerator, DistributedDataParallelKwargs

In [2]:
from huggingface_hub import notebook_login

# Handle lighting AI studio
if '/teamspace' in os.getcwd():
    os.chdir('/teamspace/studios/this_studio/llm2vec-da')
    print(os.getcwd())

notebook_login()

In [5]:
from transformers import HfArgumentParser
from llm2vec_da.arguments import SimCSEModelArguments, SimCSEDataTrainingArguments, TrainingArguments, SimCSECustomArguments

simcse_parser = HfArgumentParser(
        (SimCSEModelArguments, SimCSEDataTrainingArguments, TrainingArguments, SimCSECustomArguments)
    )

model_args, data_args, training_args, custom_args = simcse_parser.parse_json_file(
        "configs/supervised/MetaLlama3-swe-supervised-dk-curated.json"
    )

if training_args.ddp_find_unused_parameters:
    kwargs = [
        DistributedDataParallelKwargs(
            dim=0,
            broadcast_buffers=True,
            bucket_cap_mb=25,
            find_unused_parameters=True,
            check_reduction=False,
            gradient_as_bucket_view=False,
        )
    ]
else:
    kwargs = []

accelerator = Accelerator(kwargs_handlers=kwargs)
transformers.set_seed(training_args.seed)